## Preprocess

In [45]:
import xml.etree.ElementTree as ET
import re

input_file = 'test_input.xml'
output_file = 'test_inter.xml'

with open(input_file, 'r', encoding='utf-8') as input_file:
    xml_data = input_file.read()

tree = ET.ElementTree(ET.fromstring(xml_data))
root = tree.getroot()

def remove_elements_by_tag(tag_name):
    for element in root.findall(f'.//{tag_name}'):
        parent = root.find(f'.//{tag_name}/..')
        if parent is not None:
            parent.remove(element)
            
remove_elements_by_tag('GlobalAction')
remove_elements_by_tag('Story')
remove_elements_by_tag('StopTrigger')

pattern = re.compile(r'\s+(?=/>)')
root = pattern.sub('', ET.tostring(root, encoding='unicode', method='xml'))
declaration = '<?xml version="1.0" ?>\n'
root = declaration + root

with open(output_file, 'w', encoding='utf-8') as file:
    file.write(root)


## Postprocess


In [46]:
import xml.etree.ElementTree as ET

input_file = 'test_input.xml'
inter_file = 'test_inter.xml'
output_file = 'test_output.xml'

with open(input_file, 'r', encoding='utf-8') as input_file:
    input = input_file.read()

tree1 = ET.ElementTree(ET.fromstring(input))
root1 = tree1.getroot()

with open(inter_file, 'r', encoding='utf-8') as input_file:
    inter = input_file.read()

tree2 = ET.ElementTree(ET.fromstring(inter))
root2 = tree2.getroot()

def extract_and_remove_elements_by_tag(root, tag_name, parent_tag_name=None):
    parent = root.find(f'.//{parent_tag_name}')
    if parent is None: 
        return []
    elements = parent.findall(f'.//{tag_name}')
    for element in elements:
        parent.remove(element)
  
    return elements

global_actions = extract_and_remove_elements_by_tag(root1, 'GlobalAction', 'Actions')
stories = extract_and_remove_elements_by_tag(root1, 'Story', 'Storyboard')
stop_triggers = extract_and_remove_elements_by_tag(root1, 'StopTrigger', 'Storyboard')

actions = root2.find('.//Storyboard/Init/Actions')
storyboard = root2.find('.//Storyboard')
init_block = root2.find('.//Storyboard/Init')

for i, action in enumerate(global_actions):
    actions.insert(i, action)

for story in stories:
    init_index = list(storyboard).index(init_block) + 1
    storyboard.insert(init_index, story)

last_story_index = [i for i, child in enumerate(storyboard) if child.tag == 'Story'][-1]
for stop_trigger in stop_triggers:
    storyboard.insert(last_story_index + 1, stop_trigger)
    last_story_index += 1  
    
pattern = re.compile(r'\s+(?=/>)')
root = pattern.sub('', ET.tostring(root2, encoding='unicode', method='xml'))
declaration = '<?xml version="1.0" ?>\n'
root = declaration + root

with open(output_file, 'w', encoding='utf-8') as file:
    file.write(root)
